In [1]:
import email, smtplib, ssl, getpass
import pandas as pd

from email import encoders
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

In [1]:
# Startng a secure SMTP connection 

smtp_server = "smtp.SERVER.com"
port = 587  # For starttls

sender_email = "my@akp.com"
#receiver_email = "your@akp.com"
password = getpass.getpass(prompt='Password:')

message = MIMEMultipart("alternative")
message["Subject"] = "multipart test"
message["From"] = sender_email
message["To"] = receiver_email
message["Bcc"] = receiver_email  # Recommended for mass emails

# Create the plain-text and HTML version of your message
text = """\
Hi {name},
How are you?
Testing mail for Hr Automation Project"""
html = """\
<html>
  <body>
    <p>Hi {name},<br>
       How are you?<br>
       <a href="https://www.acuitykp.com/colombo/">This is a link</a> 
       Testing mail for Hr Automation Project
    </p>
  </body>
</html>
"""

# Turn these into plain/html MIMEText objects
part1 = MIMEText(text, "plain")
part2 = MIMEText(html, "html")

#add attachement 


# Add HTML/plain-text parts to MIMEMultipart message
# The email client will try to render the last part first
message.attach(part1)
message.attach(part2)

def attachFile(filepath):
    #filepath = "document.pdf"  # In same directory as script

    # Open PDF file in binary mode
    with open(filepath, "rb") as attachment:
        # Add file as application/octet-stream
        # Email client can usually download this automatically as attachment
        part = MIMEBase("application", "octet-stream")
        part.set_payload(attachment.read())

    # Encode file in ASCII characters to send by email    
    encoders.encode_base64(part)

    # Add header as key/value pair to attachment part
    part.add_header(
        "Content-Disposition",
        f"attachment; filepath= {filepath}",
    )

    # Add attachment to message and convert message to string
    message.attach(part)
    text = message.as_string()

# Create a secure SSL context
context = ssl.create_default_context()

# Try to log in to server and send email
try:
    server = smtplib.SMTP(smtp_server,port)
    server.ehlo() # Can be omitted
    server.starttls(context=context) # Secure the connection
    server.ehlo() # Can be omitted
    server.login(sender_email, password)
    
    reader = pd.read_excel('recruit_info.xlsx', index_col=None, sheet_name='Information')  
    # creating a list of dataframe columns
    for row in reader.head().itertuples():
        server.sendmail(
                sender_email,
                row.email,
                message.format(name=row.name),
            )            
    
except Exception as e:
    # Print any error messages to stdout
    print(e)
finally:
    server.quit() 

NameError: name &#39;getpass&#39; is not defined

In [52]:
# local SMTP server 
# python -m smtpd -c DebuggingServer -n localhost:1025

port = 1025  # For localhost
smtp_server = "localhost"
sender_email = input("Enter your Email: ")   # Enter your address
subject_email = input("Enter the Subject: ") 
reader = pd.read_excel('recruit_info.xlsx', index_col=None, sheet_name='Information')  
sender_info = pd.read_excel('recruit_info.xlsx', index_col=None, header=None, sheet_name='HR Information')  

# function to set message body - this sets both the html type and the fail safe of basic text 
def setMessageBody(templatePath):
    
    # Create the plain-text and HTML version of your message
    text = """\
    Hi {name},
    How are you?
    Testing mail for HR Automation Project"""
    
    #importing html email template 
    with open(templatePath, 'r') as f:
        html = f.read()

    # Turn these into plain/html MIMEText objects
    part1 = MIMEText(text, "plain")
    #part2 = MIMEText(html, "html")
    # Add HTML/plain-text parts to MIMEMultipart message
    # The email client will try to render the last part first
    message.attach(part1)
    #message.attach(part2)

# This function can be called out to attach files to the mail body
def attachFile(filepath='dummy.pdf'):
    #filepath = "document.pdf"  # In same directory as script

    # Open PDF file in binary mode
    with open(filepath, "rb") as attachment:
        # Add file as application/octet-stream
        # Email client can usually download this automatically as attachment
        part = MIMEBase("application", "octet-stream")
        part.set_payload(attachment.read())

    # Encode file in ASCII characters to send by email    
    encoders.encode_base64(part)

    # Add header as key/value pair to attachment part
    part.add_header(
        "Content-Disposition",
        f"attachment; filepath= {filepath}",
    )

    # Add attachment to message and convert message to string
    message.attach(part)
    text = message.as_string()

    

try:
    server = smtplib.SMTP(smtp_server,port)
    server.ehlo() # Can be omitted
    
    # creating a list of dataframe columns
    for row in reader.head().itertuples(): 
        
        message = MIMEMultipart("alternative")
        
        message["To"] = row.email 
        message["From"] = sender_info.iloc[0,1]
        message["Subject"] = sender_info.iloc[1,1]
             
        def templatePicker(x):
            return {
                1: 'email_templates/accept_template/index.html',
                2: 'email_templates/accept_template/index.html'
            }.get(x, 'email_templates/accept_template/index.html')              
        setMessageBody(templatePicker(row.state))
        
        #attachFile()
        
        server.sendmail(
                sender_email,
                row.email,
                message.as_string().format(name=row.name),
            )            
    
except Exception as e:
    # Print any error messages to stdout
    print(e)
finally:
    server.quit() 

Enter your Email: sdads
Enter the Subject: xxxx


In [55]:
sender_info


,0,1
0,Email Sender,hr@akp.com
1,Email Subject,Acuity Knowledge Partners Opportunities
